# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The architecture here consists of an encoder and a decoder.  The encoder uses transfer learning, where a pretrained resnet50 is connected to a fully connected layer to produce an embedding (`embed_size=300`).  The weights in the resnet are frozen while those in the fully connected layer are trainable. 

For the decoder RNN, there is a mulit-layer LSTM cell followed by a fully connected layer.  I have used 2 stacked LSTM layers.  I found it to have better performance than 1, 3 or 4 layers.  The LSTM is followed by a fully connected layer that takes the output of the LSTM and produces a vector of probabilities over the vocabulary (the most likely word to be output, could be obtained, for example, by doing an `argmax` on this vector). The output of the decoder at each time step is used as the input to the next timestep (after it is embedded into 300-element vector using `torch.nn.Embedding`.

I have also used dropout between the various LSTM cells.  I also tried a dropout layer after the LSTMs, but found that the training loss remained too high.  So, I did not used this dropout layer in the end.  

I closely consulted [this paper](https://arxiv.org/pdf/1502.03044.pdf) as a guide for the architecture and the parameters.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
The default transform train seems to do the job fine. RandomHorizontalFlip with 0.5 probability would robustify the training data in case there is some bias in the photography.  It's hard to find a good reason to have a different propability here. Regarding RandomCrop, there isn't really a choice since the input needs to be $222 \times 224$.  It's mentioned in `1_Preliminaries` that Normalize the images with the same average and standard deviation value as the training dataset (ImageNet) to make them same as the pre-trained model's distribution.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

For the encoder, the resNet50 portion is not trained. We can gain from the pre-training that was done and also, it would be enormously computationally expensive to train those weights.  Only the embedder portion of the encoder is trained, as is usual with transfer learning.  For the decoder, on the other hand, the entire network is trained, including the weights and biases within the LSTMs as well as the fully connected layer following them.  

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 
For optimizer, I chose Adam over an SGD. Using the default parameters of $\alpha=0.001, \beta_1=0.99, \beta_2=0.999$ worked well. I also tried a few other values of $\alpha$ before settling on this. Adam is often a stable optimizer and coverges faster than SGD. It combines the benefits of RMSprop and Momentum.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import os

In [2]:
## TODO #1: Select appropriate values for the Python variables below.
num_epochs = 20             # number of training epochs    
batch_size = 512          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 300          # dimensionality of image and word embeddings
hidden_size = 256          # number of features in hidden state of the RNN decoder
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
drop_ratio=0.1
num_layers=2
lr=0.001
import os
img_cap_home=os.getcwd()

In [3]:
# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         cocoapi_loc='/data/datasets', #comment this out if not running locally.
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=num_layers, drop_ratio=0.05)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(encoder.embed.parameters()) + list(decoder.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=lr )
# the other optimizer params are default - betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.55s)
creating index...


  0%|          | 2030/414113 [00:00<00:39, 10392.94it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:41<00:00, 9994.67it/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np

import requests
import time

# Open the training log file.
f = open(log_file, 'w')


#comment this out if running locally
# old_time = time.time()
# response = requests.request("GET", 
#                             "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                             headers={"Metadata-Flavor":"Google"})

# Create directory (if needed) where models will be saved
models_dirname = './models/drop{p}_b{b}_lr{lr}_hidden{h}'.format(p=drop_ratio, b=batch_size,lr=lr,h=hidden_size)
if not os.path.exists( models_dirname ):
    os.mkdir( models_dirname )  

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        #comment this out if running locally
#         if time.time() - old_time > 60:
#             old_time = time.time()
#             requests.request("POST", 
#                              "https://nebula.udacity.com/api/v1/remote/keep-alive", 
#                              headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join(models_dirname, 'decoder-epoch{ep}.pkl'.format(ep=epoch)))
        torch.save(encoder.state_dict(), os.path.join(models_dirname, 'encoder-epoch{ep}.pkl'.format(ep=epoch)))
#       torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/20], Step [100/809], Loss: 4.5313, Perplexity: 92.8801
Epoch [1/20], Step [200/809], Loss: 4.2458, Perplexity: 69.81078
Epoch [1/20], Step [300/809], Loss: 4.0970, Perplexity: 60.15886
Epoch [1/20], Step [400/809], Loss: 3.7957, Perplexity: 44.50791
Epoch [1/20], Step [500/809], Loss: 3.5724, Perplexity: 35.6022
Epoch [1/20], Step [600/809], Loss: 3.4219, Perplexity: 30.6274
Epoch [1/20], Step [700/809], Loss: 3.2228, Perplexity: 25.0974
Epoch [1/20], Step [800/809], Loss: 3.3246, Perplexity: 27.7882
Epoch [2/20], Step [100/809], Loss: 3.2034, Perplexity: 24.6152
Epoch [2/20], Step [200/809], Loss: 3.2671, Perplexity: 26.2343
Epoch [2/20], Step [300/809], Loss: 2.9367, Perplexity: 18.8544
Epoch [2/20], Step [400/809], Loss: 2.9699, Perplexity: 19.4902
Epoch [2/20], Step [500/809], Loss: 3.2462, Perplexity: 25.6919
Epoch [2/20], Step [600/809], Loss: 2.6823, Perplexity: 14.6193
Epoch [2/20], Step [700/809], Loss: 2.6627, Perplexity: 14.3353
Epoch [2/20], Step [800/809], Loss: 2

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
# import val_data_loader
from val_data_loader import val_get_loader
from model import EncoderCNN, DecoderRNN
import math
import os
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib inline
os.chdir(img_cap_home)

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def clean_sentence(output):
    i = 1
    sentence = ''
    while (i< len(output) and output[i] not in [0,1]): #If the sentence restarts, then stop.
        sentence = sentence + ' ' + data_loader.dataset.vocab.idx2word[output[i]]
        i += 1
    return sentence.strip()

In [6]:
batch_size = 1          # batch size
vocab_from_file = True    # if True, load existing vocab file
log_file = 'validation_log.txt'       # name of file with saved training loss and perplexity

In [7]:
transform_train = transforms.Compose([ 
    transforms.Resize((224,224)),                    # smaller edge of image resized to 224x224
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

data_loader = val_get_loader(transform=transform_train,
                         mode='val',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         cocoapi_loc='/data/datasets', #comment this out if not running locally.
                         vocab_from_file=vocab_from_file)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


In [8]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

import os
import torch
from model import EncoderCNN, DecoderRNN

models_dirname = 'drop{p}_b512_lr0.001_hidden256'.format(p=drop_ratio)
encoder_file = os.path.join(models_dirname, 'encoder-epoch{N}.pkl'.format(N=num_epochs))
decoder_file = os.path.join(models_dirname, 'decoder-epoch{N}.pkl'.format(N=num_epochs))
 
# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers=num_layers, drop_ratio=0.05)
decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

DecoderRNN(
  (word_embedding): Embedding(8856, 300)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.05)
  (hidden2output): Linear(in_features=256, out_features=8856, bias=True)
  (drop_layer): Dropout(p=0.05)
  (l_softm): LogSoftmax()
)

In [9]:
def predicted_caption(image):
    image = image.to(device)
    features = encoder(image).unsqueeze(1)
    output = decoder.sample(features)
#     output = decoder.beam_sample(features) #use this to see the results from beam_search
    sentence = clean_sentence(output)
    return sentence

In [10]:
def untensor(tensor):
    return int( tensor.cpu().numpy() )   

In [11]:
# Obtain sample image before and after pre-processing.
orig_image, image, true_caption, img_id, ann_id = next(iter(data_loader))
img_id = untensor( img_id )
ann_id = untensor( ann_id )
# Visualize sample image, before pre-processing.
plt.imshow(np.squeeze(orig_image))
# plt.title('example image: {cap}'.format(cap=true_caption) )
plt.title('example image: '+str(true_caption ))
plt.show()

In [12]:
captions_annFile='/data/datasets/cocoapi/annotations/captions_val2014.json'
coco_caps = COCO(captions_annFile)
annIds = coco_caps.getAnnIds(imgIds=img_id);
anns = coco_caps.loadAnns(annIds)
print('\nAnnotations available for image in validation set:\n')
coco_caps.showAnns(anns)
print( "\nPredicted caption: \n", predicted_caption(image))

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!

Annotations available for image in validation set:

A doorway view with a toilet, shower and a sink.
Picture of bathroom with shower in far corner and toilet sitting nearest to the door.
A view of a large shower with two windows near the ceiling opened slightly
A very small bathroom with a commode a stand up shower, and sink.
A bathroom with tiled floors and tiled walls. 

Predicted caption: 
 a bathroom with a toilet and a sink .


In [13]:
# Create json file for BLEU score computation
# len(data_loader.dataset.ids)
prediction_list= []
json_size=5000 #validate over 5000 random samples from the validation set
for i in range(json_size): 
    orig_image, image, true_caption, img_id, ann_id = next(iter(data_loader))
    img_id = untensor( img_id )
    dic={}
    dic["image_id"]=img_id
    dic["caption"]=predicted_caption(image)
    prediction_list.append(dic)

In [14]:
import json
pred_json_filename='./results/captions_val2014_predcap{j}_results.json'.format(j=json_size)
with open(pred_json_filename, 'w') as fout:
    json.dump(prediction_list, fout)

### Compute BLEU scores

Here, the package used for the BLEU score is [Microsoft COCO Caption Evaluation package](https://github.com/salaniz/pycocoevalcap) which is the Python 3 version of the [original repository](https://github.com/tylin/coco-caption). 

Reference papers are [this](http://www.aclweb.org/anthology/P02-1040.pdf) and [this](https://arxiv.org/pdf/1411.4555.pdf)

In [15]:
%matplotlib inline
from pycocotools.coco import COCO

In [16]:
import sys
coco_cap_dir='/data/datasets/cocoapi/coco_captions/'
sys.path.append(coco_cap_dir)
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [17]:
# set up file names and pathes
dataDir='.'
dataType='val2014'
algName = 'predcap{j}'.format(j=json_size)
# algName = 'fakecap'
annFile='%s/annotations/captions_%s.json'%(dataDir,dataType)
subtypes=['results', 'evalImgs', 'eval']
[resFile, evalImgsFile, evalFile]= \
['%s/results/captions_%s_%s_%s.json'%(dataDir,dataType,algName,subtype) for subtype in subtypes]

In [18]:
dest= coco_cap_dir + 'results/'
os.system('cp {orig} {dest}/'.format(orig=resFile, dest=dest))
os.chdir(coco_cap_dir)

In [19]:
# download Stanford models
!./get_stanford_models.sh

Found Stanford CoreNLP.


In [20]:
# create coco object and cocoRes object
coco = COCO(annFile)
cocoRes = coco.loadRes(resFile)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


In [21]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
cocoEval.params['image_id'] = cocoRes.getImgIds()

In [22]:
# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 36196, 'reflen': 35732, 'guess': [36196, 32459, 28722, 24985], 'correct': [23995, 11555, 4880, 2120]}
ratio: 1.0129855591626271
Bleu_1: 0.663
Bleu_2: 0.486
Bleu_3: 0.342
Bleu_4: 0.242
computing METEOR score...
METEOR: 0.219
computing Rouge score...
ROUGE_L: 0.491
computing CIDEr score...
CIDEr: 0.662


In [23]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print( '%s: %.3f'%(metric, score))
os.chdir(img_cap_home)

Bleu_1: 0.663
Bleu_2: 0.486
Bleu_3: 0.342
Bleu_4: 0.242
METEOR: 0.219
ROUGE_L: 0.491
CIDEr: 0.662


### Comments on the scores

BLEU, Bilingual Evaluation Understudy, introduced by [Papineni et al](http://aclweb.org/anthology/P/P02/P02-1040.pdf) in 2002 is one metric to evaluate translations. Although it has its limitations (see below), it has become the standard.  The fundamental idea is to compare outputs of a machine translation system with human generated reference translations.

Bleu_1 corresponds to *adequacy* and higher order Bleu scores correspond to *fluency*.

Here, with 20 epochs of training, the 1-gram BLEU score of 0.663 on the validation set outperforms the 0.655 attained in [this 2018 study](https://arxiv.org/pdf/1801.05568.pdf) (although the study does not specify which n-gram or combination of them it is using.). 

__Some drawbacks of BLEU__

- Even perfect translations may get penalized by BLEU if they're distinct from the reference translations.
- BLEU is not good at measuring the overall grammar or even word order within candidates, since it's mainly check on cooccurrence of words between candidate sentences and reference translations.

Nevertheless, since errors smooth out when BLEU is performed over entire corpuses, it's a very good tool to compare various translation models (or in this case, image captioning models).
